# Spotify Final Project - Emily Xie EDA
### Milestone 3: EDA and Baseline Model
### Austin Rochon, Emily Xie, and Mark Lock

In [48]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns




## Setting up to query Google BigQuery

Since the data set is 70 million rows, and cannot all be easily managed on our local machines, we've decided to use GCP (Google Cloud Developer) platform to help us manage our data. First, we uploaded the CSVs into a private GCP  storage bucket, and from there sent the data to BigQuery, GCP's distributed database, which will allow us to query the data however we like.

Here's the interface (you need access): https://console.cloud.google.com/bigquery?folder=&organizationId=&project=spotted-d&p=spotted-d&d=playlist_songs&t=playlist_songs&page=table

From here, we can retrieve, query, join, and manipulate the entirety of the dataset however we like. Please follow the instructions below in this notebook to set up your Jupyter so that it can query directly from GCP. 

### Setup Instructions

The following setup should take just a few minutes.

#### 1. Create a service account. 
Go to https://console.cloud.google.com/iam-admin/serviceaccounts?project=spotted-d. Once you are on this page, go to the "actions" tab, where there will be a drop-down indicated by three dots on the right-most part of the corresponding account. Click "Create Key" which will download a key for you somewhere on your local machine. Save it somewhere safe. :-)  

If you plan to store it on this git project, make sure to put in a folder that is git-ignored so that we don't push it up to Github. I actually created a .gitignore file on this branch. If you create a directory called "config" under the top-level directory of this repository and stick your service key in there, it should be automatically ignored. Ask me if you want any help.

#### 2. Set up implicit authentication with gCloud
If you are using Mac, just run this on your command line:

`export GOOGLE_APPLICATION_CREDENTIALS="[PATH]"`

If you are using Windows:

`$env:GOOGLE_APPLICATION_CREDENTIALS="[PATH]"`

For more info, see below:

https://cloud.google.com/docs/authentication/getting-started#auth-cloud-implicit-python

#### 3. Install Google Cloud Big Query Pandas python package.

Run this on your terminal:

`pip install --upgrade google-cloud-bigquery[pandas]`

For more on the above two steps, you can check out the GCP documentation: https://cloud.google.com/bigquery/docs/visualize-jupyter

#### Run this cell to load google cloud bigquery:

In [1]:
%load_ext google.cloud.bigquery

#### Now, you can query your data. Next to "bigquery", write the name of the variable you'd like to save your query results into.

In [9]:
%%bigquery playlist_data
select ps.pid, ps.pos, ps.artist_name, ps.track_name, ps.album_name, th.*
from playlist_songs.playlist_songs as ps
inner join playlist_songs.tracks_headers as th on th.trackid = ps.track_uri
limit 100000

#### You'll notice that it automatically saves to a Pandas dataframe for you

In [10]:
len(playlist_data)

100000

## EDA

In [11]:
playlist_data.head()

,pid,pos,artist_name,track_name,album_name,index,danceability,energy,key,loudness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,trackid
0,37,126,"Harry Connick, Jr.",What a Night!,What A Night! A Christmas Album,4,0.609,0.329,3,-10.762,...,0.498,129.676,audio_features,3379BhHoOyWDkewjzpgZAv,spotify:track:3379BhHoOyWDkewjzpgZAv,https://api.spotify.com/v1/tracks/3379BhHoOyWD...,https://api.spotify.com/v1/audio-analysis/3379...,203573,4,spotify:track:3379BhHoOyWDkewjzpgZAv
1,57,58,Jack Johnson,Change,From Here To Now To You,64,0.752,0.405,11,-8.414,...,0.321,92.975,audio_features,4WAoUYToqNAC6AGGslFxeo,spotify:track:4WAoUYToqNAC6AGGslFxeo,https://api.spotify.com/v1/tracks/4WAoUYToqNAC...,https://api.spotify.com/v1/audio-analysis/4WAo...,194053,4,spotify:track:4WAoUYToqNAC6AGGslFxeo
2,73,156,Bon Iver,____45_____,"22, A Million",29,0.410,0.121,2,-15.473,...,0.177,86.359,audio_features,14iePHBA4OZzeMp0gtMehM,spotify:track:14iePHBA4OZzeMp0gtMehM,https://api.spotify.com/v1/tracks/14iePHBA4OZz...,https://api.spotify.com/v1/audio-analysis/14ie...,166107,3,spotify:track:14iePHBA4OZzeMp0gtMehM
3,122,137,Counting Crows,Rain King,August And Everything After,78,0.492,0.879,9,-6.723,...,0.750,157.669,audio_features,7s4JCyY8YIZSsKvtwStBQ6,spotify:track:7s4JCyY8YIZSsKvtwStBQ6,https://api.spotify.com/v1/tracks/7s4JCyY8YIZS...,https://api.spotify.com/v1/audio-analysis/7s4J...,255693,4,spotify:track:7s4JCyY8YIZSsKvtwStBQ6
4,141,25,Silverstein,Defend You,Discovering the Waterfront (Reissue),29,0.327,0.988,4,-2.246,...,0.353,198.070,audio_features,7lpubMQy7uIVqltp6wVCpo,spotify:track:7lpubMQy7uIVqltp6wVCpo,https://api.spotify.com/v1/tracks/7lpubMQy7uIV...,https://api.spotify.com/v1/audio-analysis/7lpu...,208173,4,spotify:track:7lpubMQy7uIVqltp6wVCpo


In [ ]:
trackcounts_by_playlist = playlists[["pid", "trackid"]].groupby(["pid"]).agg(["count"])

sns.set(color_codes=True)
sns.distplot(trackcounts_by_playlist, kde=False);
